## Data Ingestion stage

In [21]:
from collections import namedtuple
from pathlib import Path
import os

In [22]:
%pwd

'c:\\Users\\vishpand\\Desktop\\campusx DS course\\14) projects\\dog-cat-classification-e2e-deep-learning-db\\research'

In [23]:
os.chdir('../')

In [24]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [26]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [28]:
import os
import urllib.request as request
from zipfile import ZipFile

In [29]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not Path(self.config.local_data_file).exists():
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: Path):
        target_filepath = Path(working_dir) /  f
        if not Path(target_filepath).exists():
            zf.extract(f, working_dir)
        
        if Path(target_filepath).stat().st_size == 0:
            Path(target_filepath).unlink()

    

    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [33]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2024-07-10 02:11:18,677: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-10 02:11:18,679: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-10 02:11:18,681: INFO: common: created directory at: artifacts]
[2024-07-10 02:11:18,682: INFO: common: created directory at: artifacts/data_ingestion]


In [34]:
%pwd

'c:\\Users\\vishpand\\Desktop\\campusx DS course\\14) projects\\dog-cat-classification-e2e-deep-learning-db'

In [35]:
'''
x/y/Cat/1.jpg
x/y/1.jpg
'''

'\nx/y/Cat/1.jpg\nx/y/1.jpg\n'